<a href="https://colab.research.google.com/github/akash-verma22/MPI_Assignments/blob/main/Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile assign6.cu
#include <stdio.h>
#include <math.h>
#include <cuda_runtime.h>
#include <chrono>


__global__ void computeSqrt(const float *A, float *C, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N) {
        C[idx] = sqrtf(A[idx]);
    }
}


void runSqrtKernel(int N) {
    float *h_A = new float[N];
    float *h_C = new float[N];


    for (int i = 0; i < N; ++i)
        h_A[i] = static_cast<float>(i + 1);

    float *d_A, *d_C;
    cudaMalloc(&d_A, N * sizeof(float));
    cudaMalloc(&d_C, N * sizeof(float));

    cudaMemcpy(d_A, h_A, N * sizeof(float), cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocks = (N + threadsPerBlock - 1) / threadsPerBlock;


    auto start = std::chrono::high_resolution_clock::now();

    computeSqrt<<<blocks, threadsPerBlock>>>(d_A, d_C, N);
    cudaDeviceSynchronize();

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> elapsed = end - start;

    printf("N = %d, Time = %f ms\n", N, elapsed.count());

    cudaMemcpy(h_C, d_C, N * sizeof(float), cudaMemcpyDeviceToHost);


    delete[] h_A;
    delete[] h_C;
    cudaFree(d_A);
    cudaFree(d_C);
}

int main() {
    const int numTests = 4;
    int sizes[numTests] = {50000, 500000, 5000000, 50000000};



    for (int i = 0; i < 4; ++i) {

        runSqrtKernel(sizes[i]);
    }

    return 0;
}


Writing assign6.cu


In [2]:
!nvcc -O2 -o assign6 assign6.cu
!./assign6

N = 50000, Time = 61.699779 ms
N = 500000, Time = 0.079040 ms
N = 5000000, Time = 0.076638 ms
N = 50000000, Time = 0.061090 ms


In [3]:
!pip install pandas xlsxwriter openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.2 MB/s eta 0:00:00


In [4]:
import pandas as pd


data = {
    'Array Size': [50000, 500000, 5000000, 50000000],
    'Time (ms)': [7.6290,0.0950,0.0706,0.0618]
}


df = pd.DataFrame(data)


with pd.ExcelWriter('results.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Timings', index=False)

    workbook  = writer.book
    worksheet = writer.sheets['Timings']


    chart = workbook.add_chart({'type': 'line'})


    chart.add_series({
        'name':       'Execution Time',
        'categories': ['Timings', 1, 0, len(df), 0],
        'values':     ['Timings', 1, 1, len(df), 1],
        'marker':     {'type': 'circle', 'size': 5}
    })


    chart.set_title({'name': 'CUDA Square Root Kernel Performance'})
    chart.set_x_axis({'name': 'Array Size'})
    chart.set_y_axis({'name': 'Execution Time (ms)'})
    chart.set_style(10)


    worksheet.insert_chart('D2', chart)
